In [1]:
import pandas as pd
import numpy as np

In [16]:
impressions_df_1 = pd.read_csv("../datasets/impressions_q1_extended_v2.csv")
impressions_df_2 = pd.read_csv("../datasets/impressions_q2_extended_v2.csv")
#impressions_df_1 = pd.read_csv("../datasets/impressions_q3_extended_v2.csv")
#impressions_df_2 = pd.read_csv("../datasets/impressions_q4_extended_v2.csv")

#impressions_df_2 = pd.read_csv("../datasets/impressions_02_extended_v2.csv")
impressions_df = pd.concat([impressions_df_1, impressions_df_2])
#client_df = pd.read_excel("../datasets/report - 2023-02-01T152103.748 (1).xlsx")
nordics_df = pd.read_excel("../datasets/Schibsted_Placement_Viewability.xlsx")

In [17]:
import json

def extract_pid(placement_id):
    try:
        placement_id = json.loads(placement_id)
    except:
        placement_id = {"tag_id": [""], "bam_ad_slot": []}
    bam_ad_slots = placement_id["bam_ad_slot"]
    # print(bam_ad_slots)
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    ret = ''
    try:
        ret = placement_id['tag_id'][0]
    except:
        # print('.')
        ret = ','.join([bas for bas in final_bam_ad_slots])
    else:
        ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret
    

def len_pids(placement_id):
    try:
        placement_id = json.loads(placement_id)
    except:
        placement_id = {"tag_id": [""], "bam_ad_slot": []}
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    return len(placement_id["tag_id"]) + len(final_bam_ad_slots)

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_len'] = impressions_df['placement_ids'].apply(len_pids)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)

In [18]:
impressions_df = impressions_df[impressions_df["pid_type"] == 'int']
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))
#all_imps_for_q4 = impressions_df.copy()

In [ ]:
#schibsted_impressions_df = impressions_df[impressions_df["pid"].isin(list(client_df["Placement Id"]))]
nordics_impressions_df = impressions_df[impressions_df["pid"].isin(list(set(nordics_df["Placement Id"])))]
#impressions_df = nordics_impressions_df.copy()

In [ ]:
nordics_impressions_df

In [19]:
impressions_df = impressions_df[impressions_df["pid"].isin(list(nordics_df["Placement Id"]))]
impressions_df = impressions_df[impressions_df["adblock_detected"] == False]
impressions_df["is_duration_inview"] = impressions_df.apply(lambda x: True if x["exist_viewable_1_s_threshold_50"] == True else False, axis=1)
impressions_df["true_is_iab_inview"] = impressions_df.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)
impressions_df

,id,hostname,installation_id,session_id,beacon_type_chosen,ad_technical_format,channel,chosen_brand,ad_width_chosen,ad_height_chosen,...,time_in_screen,scroll_speed,total_fixation_duration,part_year,part_month,pid,pid_len,pid_type,is_duration_inview,true_is_iab_inview
27,4be35c01-a401-49f7-a8df-18e4c56f8663,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,7d2c1c95-b9c8-4a7e-9c39-ff7142a93815,ad/adnxs.com-iframe-v3,display,display,pliing,500,555,...,NaN,NaN,0,2022,3,19499431,1,int,True,True
28,5391c8f8-680a-4d38-a4c1-c6883582e889,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,7d2c1c95-b9c8-4a7e-9c39-ff7142a93815,ad/adnxs.com-a-img-v1,display,display,samsung,250,480,...,NaN,NaN,0,2022,3,19499423,1,int,True,True
29,1e54ab6d-ae89-46b8-baee-85b9160a8e2e,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,7d2c1c95-b9c8-4a7e-9c39-ff7142a93815,ad/adnxs.com-iframe-v3,display,display,samsung,250,480,...,NaN,NaN,0,2022,3,19499423,1,int,True,True
30,cea91fe6-0bb4-44ff-a0d8-4c5b2a376767,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,7d2c1c95-b9c8-4a7e-9c39-ff7142a93815,ad/doubleclick.net-ins-dcmads-v1,display,display,adnxs,250,480,...,NaN,NaN,0,2022,3,19499424,1,int,True,True
31,8d55588d-3b29-4b65-901c-f669ac538007,aftonbladet.se,a4d6b1bc-642e-4ea4-bf8d-1687fcba935e,7d2c1c95-b9c8-4a7e-9c39-ff7142a93815,ad/doubleclick.net-a-pcs-click-img-v1,display,display,adnxs,250,480,...,NaN,NaN,0,2022,3,19499424,1,int,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569554,a2a04d01-d6fd-44ce-80cb-a39b1cbf24c0,blocket.se,af9b036a-ba2f-4a81-9f64-db6ca75caea3,4d0e2ce4-58f6-45d0-9dc5-eb2fb036c09d,ad/doubleclick.net-a-pcs-click-img-v1,display,display,toyota,160,600,...,NaN,NaN,0,2022,5,19499533,1,int,True,True
4569596,6e1802ab-110d-47c6-88fc-3fd9abaa40a1,aftonbladet.se,737b86fb-5db9-4d07-b5bd-5419676cb4d8,37fd06f0-a3f4-4ea0-bc86-9069f0112655,ad/anxv,display,display,adnxs,180,150,...,NaN,NaN,0,2022,5,19499441,1,int,False,False
4569616,6ad8f5fd-ef8d-42bf-ac6d-926ddb5b9a34,aftonbladet.se,a7ec5df5-369f-4f14-8db8-3fa0076bb9e3,784c2271-0a77-4bf5-8530-89d332368711,ad/adnxs.com-a-img-v1,display,display,adnxs,250,480,...,NaN,NaN,0,2022,5,19499423,1,int,False,False
4569620,59d9d6d1-52fe-427a-80e7-b4fd4372ed58,blocket.se,d5b364b9-3dd2-4c16-a529-a143712f508d,7bf71a5b-9733-47e3-83b9-0006ed6f7047,ad/tagid-iframe-v1,display,display,avanza,1180,240,...,NaN,NaN,5121,2022,5,19499521,1,int,True,True


In [ ]:
impressions_df["is_duration_inview"].value_counts()

In [ ]:
impressions_df["true_is_iab_inview"].value_counts()

In [20]:
#result_df = result_df[["hostname", "Placement Id", "Viewability Measurement Rate %", "Viewability Rate % Xandr", "Viewability Rate % Tobii", "in_view_duration_1second%", "true_in_view_iab%"]]

grouped_df = impressions_df.groupby(["hostname", "pid"]).apply(
    lambda x: pd.Series(
        {
            "num_impressions": x["id"].count(),
            #"in_view_iab": x[x["is_iab_inview"] == True]["id"].count(),
            "true_in_view_iab": x[x["true_is_iab_inview"] == True]["id"].count(),
            #"in_view_duration_1_second": np.sum(x["exist_viewable_1_s_threshold_50"]),
        }
    )
)
grouped_df = grouped_df.reset_index()
grouped_df

,hostname,pid,num_impressions,true_in_view_iab
0,aftonbladet.se,19245836,426,161
1,aftonbladet.se,19246168,3,1
2,aftonbladet.se,19499407,37392,23339
3,aftonbladet.se,19499410,41558,26428
4,aftonbladet.se,19499411,3623,1771
...,...,...,...,...
210,wellness.se,20906194,1,1
211,wellness.se,20906231,1,1
212,wellness.se,20906246,1,1
213,wellness.se,20906247,146,103


In [ ]:
#grouped_df["in_view_iab%"] = grouped_df.apply(lambda x: round(x["in_view_iab"] / x["num_impressions"] * 100, 0), axis=1)
#grouped_df["in_view_duration_1second%"] = grouped_df.apply(lambda x: round(x["in_view_duration_1_second"] / x["num_impressions"] * 100, 0), axis=1)
grouped_df["true_in_view_iab%"] = grouped_df.apply(lambda x: round(x["true_in_view_iab"] / x["num_impressions"] * 100, 0), axis=1)
grouped_df

In [21]:
final_df = grouped_df[["hostname", "pid", "num_impressions", "true_in_view_iab"]]
final_df

,hostname,pid,num_impressions,true_in_view_iab
0,aftonbladet.se,19245836,426,161
1,aftonbladet.se,19246168,3,1
2,aftonbladet.se,19499407,37392,23339
3,aftonbladet.se,19499410,41558,26428
4,aftonbladet.se,19499411,3623,1771
...,...,...,...,...
210,wellness.se,20906194,1,1
211,wellness.se,20906231,1,1
212,wellness.se,20906246,1,1
213,wellness.se,20906247,146,103


In [22]:
result_df = pd.merge(left=nordics_df, right=final_df, how="outer", left_on="Placement Id", right_on="pid")
result_df

,Seller Name,Placement Id,Size,Imps,Viewability Rate %,hostname,pid,num_impressions,true_in_view_iab
0,Schibsted Sweden,14745815,320x320,173498,0.396244,bloggar.aftonbladet.se,14745815.0,4.0,0.0
1,Schibsted Sweden,14746054,640x320,20857,0.529030,bloggar.aftonbladet.se,14746054.0,246.0,142.0
2,Schibsted Sweden,14746054,640x320,20857,0.529030,tramsfrans.aftonbladet.se,14746054.0,4.0,0.0
3,Schibsted Sweden,14746055,640x320,7565,0.341722,bloggar.aftonbladet.se,14746055.0,257.0,112.0
4,Schibsted Sweden,14746055,640x320,7565,0.341722,tramsfrans.aftonbladet.se,14746055.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...
852,Schibsted Sweden,27719499,320x400,525,0.706000,NaN,NaN,NaN,NaN
853,Schibsted Sweden,27719499,320x320,6919,0.626704,NaN,NaN,NaN,NaN
854,Schibsted Sweden,28216731,320x320,23218,0.722889,NaN,NaN,NaN,NaN
855,Schibsted Sweden,28216731,320x160,22839,0.718606,NaN,NaN,NaN,NaN


In [23]:
result_df["Viewability Rate %"] = round(result_df["Viewability Rate %"] * 100, 0)

In [24]:
impressions_df[impressions_df["pid"] == 14745815]

,id,hostname,installation_id,session_id,beacon_type_chosen,ad_technical_format,channel,chosen_brand,ad_width_chosen,ad_height_chosen,...,time_in_screen,scroll_speed,total_fixation_duration,part_year,part_month,pid,pid_len,pid_type,is_duration_inview,true_is_iab_inview
4272283,5ddcf663-a130-4d3b-9c37-7432a5682688,bloggar.aftonbladet.se,41b63ca2-9273-478c-ba8f-5628e24299e7,7bb72eba-97b1-4556-8ff2-eac75484a966,ad/adnxs.com-iframe-v3,display,display,adnxs-simple,300,250,...,NaN,NaN,0,2022,1,14745815,1,int,False,False
4272284,e394923a-df80-4741-8dfd-d06bd97165a0,bloggar.aftonbladet.se,41b63ca2-9273-478c-ba8f-5628e24299e7,7bb72eba-97b1-4556-8ff2-eac75484a966,ad/adnxs.com-iframe-v3,display,display,maria,300,250,...,NaN,NaN,0,2022,1,14745815,1,int,False,False
4918420,b0133a1c-5484-4abc-8e8f-bc621b91217e,bloggar.aftonbladet.se,41b63ca2-9273-478c-ba8f-5628e24299e7,7bb72eba-97b1-4556-8ff2-eac75484a966,ad/adnxs.com-iframe-v3,display,display,maria,300,250,...,NaN,NaN,0,2022,1,14745815,1,int,False,False
4918767,5a147c8f-9bc4-4509-8a33-e7e697d62c66,bloggar.aftonbladet.se,41b63ca2-9273-478c-ba8f-5628e24299e7,7bb72eba-97b1-4556-8ff2-eac75484a966,ad/tagid-iframe-v1,display,display,adnxs,300,250,...,NaN,NaN,0,2022,1,14745815,1,int,False,False


In [25]:
result_df = result_df.dropna()

In [26]:
result_df = result_df.astype({"pid": "int64", "num_impressions": "int64"})
result_df

,Seller Name,Placement Id,Size,Imps,Viewability Rate %,hostname,pid,num_impressions,true_in_view_iab
0,Schibsted Sweden,14745815,320x320,173498,40.0,bloggar.aftonbladet.se,14745815,4,0.0
1,Schibsted Sweden,14746054,640x320,20857,53.0,bloggar.aftonbladet.se,14746054,246,142.0
2,Schibsted Sweden,14746054,640x320,20857,53.0,tramsfrans.aftonbladet.se,14746054,4,0.0
3,Schibsted Sweden,14746055,640x320,7565,34.0,bloggar.aftonbladet.se,14746055,257,112.0
4,Schibsted Sweden,14746055,640x320,7565,34.0,tramsfrans.aftonbladet.se,14746055,4,0.0
...,...,...,...,...,...,...,...,...,...
814,Schibsted Sweden,23373701,640x320,39492,65.0,aftonbladet.se,23373701,178,51.0
815,Schibsted Sweden,23375557,640x320,216211,76.0,aftonbladet.se,23375557,869,430.0
816,Schibsted Sweden,23375567,320x320,863697,75.0,aftonbladet.se,23375567,2,1.0
817,Schibsted Sweden,23582491,250x600,7305,95.0,malservice.aftonbladet.se,23582491,182,139.0


In [27]:
result_df.to_excel("../temp/schibsted_viewability_q1_q2.xlsx", index=False)

In [ ]:
result_df = result_df[["hostname", "Placement Id", "Placement Name", "Viewability Measurement Rate %", "Viewability Rate % Xandr", "Viewability Rate % Tobii", "true_in_view_iab%"]]
result_df

In [ ]:
result_df.rename(columns={"true_in_view_iab%": "Viewability rate % Tobii True IAB"}, inplace=True)
result_df

In [ ]:
result_df = result_df[result_df["hostname"].notna()]
result_df["Placement Id"] = result_df["Placement Id"].round().astype(int)
result_df["Viewability Rate % Tobii"] = result_df["Viewability Rate % Tobii"].round().astype(int)
result_df["Viewability rate % Tobii 1 second"] = result_df["Viewability rate % Tobii 1 second"].round().astype(int)
result_df["Viewability rate % Tobii IAB or 1 second"] = result_df["Viewability rate % Tobii IAB or 1 second"].round().astype(int)
result_df["Viewability Measurement Rate %"] = result_df.apply(lambda x: round(x["Viewability Measurement Rate %"] * 100, 0), axis=1)
result_df["Viewability Rate % Xandr"] = result_df.apply(lambda x: round(x["Viewability Rate % Xandr"] * 100, 0), axis=1)old_placements = 

result_df

In [ ]:
result_df["Viewability Measurement Rate %"] = result_df["Viewability Measurement Rate %"].round().astype(int)
result_df["Viewability Rate % Xandr"] = result_df["Viewability Rate % Xandr"].round().astype(int)

result_df

In [ ]:
#result_df[result_df["Viewability Rate % Tobii"] > result_df["Viewability rate % Tobii IAB or 1 second"]]

In [ ]:
#result_df.to_excel("../final_reports/ATEX2-966.xlsx")
#result_df.to_csv("../final_reports/ATEX2-966.csv")

In [36]:
df_1 = pd.read_excel('../temp/schibsted_viewability_q1_q2.xlsx')
df_2 = pd.read_excel('../temp/schibsted_viewability_q3_q4.xlsx')
merged_df = pd.merge(left=df_1, right=df_2, how="outer", on="Placement Id", indicator=True)
merged_df

,Seller Name_x,Placement Id,Size_x,Imps_x,Viewability Rate %_x,hostname_x,pid_x,num_impressions_x,true_in_view_iab_x,Seller Name_y,Size_y,Imps_y,Viewability Rate %_y,hostname_y,pid_y,num_impressions_y,true_in_view_iab_y,_merge
0,Schibsted Sweden,14745815,320x320,173498.0,40.0,bloggar.aftonbladet.se,14745815.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,Schibsted Sweden,14746054,640x320,20857.0,53.0,bloggar.aftonbladet.se,14746054.0,246.0,142.0,Schibsted Sweden,640x320,20857.0,53.0,bloggar.aftonbladet.se,14746054.0,107.0,44.0,both
2,Schibsted Sweden,14746054,640x320,20857.0,53.0,tramsfrans.aftonbladet.se,14746054.0,4.0,0.0,Schibsted Sweden,640x320,20857.0,53.0,bloggar.aftonbladet.se,14746054.0,107.0,44.0,both
3,Schibsted Sweden,14746055,640x320,7565.0,34.0,bloggar.aftonbladet.se,14746055.0,257.0,112.0,Schibsted Sweden,640x320,7565.0,34.0,bloggar.aftonbladet.se,14746055.0,107.0,36.0,both
4,Schibsted Sweden,14746055,640x320,7565.0,34.0,tramsfrans.aftonbladet.se,14746055.0,4.0,0.0,Schibsted Sweden,640x320,7565.0,34.0,bloggar.aftonbladet.se,14746055.0,107.0,36.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,NaN,27264109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schibsted Sweden,320x400,63.0,72.0,svd.se,27264109.0,101.0,51.0,right_only
1494,NaN,27264109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schibsted Sweden,320x320,4024.0,49.0,svd.se,27264109.0,101.0,51.0,right_only
1495,NaN,27264940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schibsted Sweden,320x400,3479.0,69.0,svd.se,27264940.0,17.0,9.0,right_only
1496,NaN,27264940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schibsted Sweden,320x320,45048.0,62.0,svd.se,27264940.0,17.0,9.0,right_only


In [54]:
def sum_imps(row):
    if row["_merge"] == 'both':
        return row["Imps_x"] + row["Imps_y"]
    elif row["_merge"] == 'left_only':
        return row["Imps_x"]
    return row["Imps_y"]

def sum_num_imps(row):
    if row["_merge"] == 'both':
        return row["num_impressions_x"] + row["num_impressions_y"]
    elif row["_merge"] == 'left_only':
        return row["num_impressions_x"]
    return row["num_impressions_y"]

def get_seller(row):
    if row["_merge"] in ('both', 'left_only'):
        return row["Seller Name_x"]
    return row["Seller Name_y"]

def get_size(row):
    if row["_merge"] in ('both', 'left_only'):
        return row["Size_x"]
    return row["Size_y"]

def get_hostname(row):
    if row["_merge"] in ('both', 'left_only'):
        return row["hostname_x"]
    return row["hostname_y"]

def sum_inview(row):
    if row["_merge"] == 'both':
        return row["true_in_view_iab_x"] + row["true_in_view_iab_y"]
    elif row["_merge"] == 'left_only':
        return row["true_in_view_iab_x"]
    return row["true_in_view_iab_y"]

merged_df["sum_of_imps"] = merged_df.apply(lambda x: sum_imps(x), axis=1)
merged_df["sum_of_num_imps"] = merged_df.apply(lambda x: sum_num_imps(x), axis=1)
merged_df["final_seller"] = merged_df.apply(lambda x: get_seller(x), axis=1)
merged_df["final_hostname"] = merged_df.apply(lambda x: get_hostname(x), axis=1)
merged_df["final_size"] = merged_df.apply(lambda x: get_size(x), axis=1)
merged_df["sum_inview"] = merged_df.apply(lambda x: sum_inview(x), axis=1)

merged_df

,Seller Name_x,Placement Id,Size_x,Imps_x,Viewability Rate %_x,hostname_x,pid_x,num_impressions_x,true_in_view_iab_x,Seller Name_y,...,pid_y,num_impressions_y,true_in_view_iab_y,_merge,sum_of_imps,sum_of_num_imps,final_seller,final_hostname,final_size,sum_inview
0,Schibsted Sweden,14745815,320x320,173498.0,40.0,bloggar.aftonbladet.se,14745815.0,4.0,0.0,NaN,...,NaN,NaN,NaN,left_only,173498.0,4.0,Schibsted Sweden,bloggar.aftonbladet.se,320x320,0.0
1,Schibsted Sweden,14746054,640x320,20857.0,53.0,bloggar.aftonbladet.se,14746054.0,246.0,142.0,Schibsted Sweden,...,14746054.0,107.0,44.0,both,41714.0,353.0,Schibsted Sweden,bloggar.aftonbladet.se,640x320,186.0
2,Schibsted Sweden,14746054,640x320,20857.0,53.0,tramsfrans.aftonbladet.se,14746054.0,4.0,0.0,Schibsted Sweden,...,14746054.0,107.0,44.0,both,41714.0,111.0,Schibsted Sweden,tramsfrans.aftonbladet.se,640x320,44.0
3,Schibsted Sweden,14746055,640x320,7565.0,34.0,bloggar.aftonbladet.se,14746055.0,257.0,112.0,Schibsted Sweden,...,14746055.0,107.0,36.0,both,15130.0,364.0,Schibsted Sweden,bloggar.aftonbladet.se,640x320,148.0
4,Schibsted Sweden,14746055,640x320,7565.0,34.0,tramsfrans.aftonbladet.se,14746055.0,4.0,0.0,Schibsted Sweden,...,14746055.0,107.0,36.0,both,15130.0,111.0,Schibsted Sweden,tramsfrans.aftonbladet.se,640x320,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,NaN,27264109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schibsted Sweden,...,27264109.0,101.0,51.0,right_only,63.0,101.0,Schibsted Sweden,svd.se,320x400,51.0
1494,NaN,27264109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schibsted Sweden,...,27264109.0,101.0,51.0,right_only,4024.0,101.0,Schibsted Sweden,svd.se,320x320,51.0
1495,NaN,27264940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schibsted Sweden,...,27264940.0,17.0,9.0,right_only,3479.0,17.0,Schibsted Sweden,svd.se,320x400,9.0
1496,NaN,27264940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Schibsted Sweden,...,27264940.0,17.0,9.0,right_only,45048.0,17.0,Schibsted Sweden,svd.se,320x320,9.0


In [45]:
another_one = 

both          1442
right_only      38
left_only       18
Name: _merge, dtype: int64

In [40]:
merged_df["Placement Id"].value_counts()

14746059    168
19499712    100
20334189     96
20334190     96
20334182     64
           ... 
19499453      1
19499447      1
19499438      1
19499437      1
27264953      1
Name: Placement Id, Length: 211, dtype: int64

In [39]:
def get_seller_name(row):
    print(row["_merge"])
    if row["_merge"] in ["left_only", "both"]:
        return row["Seller Name_x"] 
    else:
        return row["Seller Name_y"]
def get_size(row):
    return row["Size_x"] or row["Size_y"]
def get_imps(row):
    return row["Imps_x"] + row["Imps_y"]
def get_hostname(row):
    return row["hostname_x"] or row["hostname_y"]
def get_num_imps(row):
    return row["num_impressions_x"] + row["num_impressions_y"]
def get_in_view(row):
    return row["true_in_view_iab_x"] + row["true_in_view_iab_y"]
def get_viewability(row):
    return row["Viewability Rate %_x"] or row["Viewability Rate %_y"]
result_df = merged_df.groupby("Placement Id").apply(
    lambda x: pd.Series({
        "Seller Name": get_seller_name(x),
        "Size": get_size(x),
        "Imps": get_imps(x),
        "Viewability Rate %": get_viewability(x),
        "hostname": get_hostname(x),
        "num_impressions": get_num_imps(x),
        "in_view": get_in_view(x)
    })
)
result_df

0    left_only
Name: _merge, dtype: category
Categories (3, object): ['left_only', 'right_only', 'both']


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().